# Optimized energy windows

This notebook uses the optimal energy windows as calculated in the low activity notebook to try and reduce the noise in the efficiency for the lowest-activity samples

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
plt.close('all')
matplotlib.use('TkAgg')

import scipy.optimize as opt
import scipy

import xlwings as xw
import os
import seaborn as sns
import glob

n = 10 # number of vials

## What are we doing here
1. Calculate new counts
+ Collect all spectra into 1 array
+ Subtract background
+ Collect timestamps
+ Calculate counts in new windows

2. Calculate efficiency
+ Collect masses
+ calculate theoretical activity
+ calculate efficiency

3. Evaluate performance
+ Fitting
+ Compare to previous estimate (in 400-600 keV window)

## 1. Calculate new counts

In [9]:
files = []
for name in glob.glob('./data/HidexAMG-Track_30min*'):
    files.append(name)

# walk through the folder with the files and collect the spectra
#for _, _, files in os.walk("./data/HidexAMG-Track_30min*"): break
files = list(filter(lambda k: '~' not in k, files)) # remove temporary files to ensure size allocation is correct for spectra and timestamps

excel_app = xw.App(visible=False)

spectra = np.empty(shape=(len(files)*n, 2048)) # size: number of total runs (files x vials) by number of energy channels
spectra.fill(np.nan)

timestamps = np.empty(shape=(len(files)*n), dtype='datetime64[s]')
#timestamps.fill(np.NaT)

i = 0
for f in files:
    #if "~" in f: continue # disregards temporary files
    print(f)
    
    # extract the correct spectra from the file
    path = f
    wb = xw.Book(path)
    sheet = wb.sheets['Spectra']
    s = sheet["C20"].options(np.array, expand='table').value
    
    # save the spectra to the array
    spectra[i*n:(i*n)+n, :] = s

    # extract the timestamps
    ws = wb.sheets['Results']
    t = ws["C27:C36"].options(np.array, dtype='datetime64[s]').value
    

    # save the timestamps to an array
    timestamps[i*n:(i*n)+n] = t
    
    wb.save()
    wb.close()

    i += 1

print("Done.")
print("Sanity check: Any NaNs in the spectra array?", np.isnan(np.sum(spectra)))
excel_app.quit()


# remove background from all spectra
bg_spectrum = np.loadtxt(r"C:\Users\yuliy\OneDrive\Documents\Fall 2023\Research\efficiency\low_activity\background.txt")
spectra -= bg_spectrum

# start the clock at 0
#timestamps -= timestamps[0]
timestamps

./data\HidexAMG-Track_30min-035-20240501-184615-AutoExport.xlsx
./data\HidexAMG-Track_30min-035-20240501-190200-AutoExport.xlsx
./data\HidexAMG-Track_30min-035-20240501-191748-AutoExport.xlsx
./data\HidexAMG-Track_30min-035-20240501-193337-AutoExport.xlsx
./data\HidexAMG-Track_30min-035-20240501-194944-AutoExport.xlsx
./data\HidexAMG-Track_30min-035-20240501-200533-AutoExport.xlsx
./data\HidexAMG-Track_30min-035-20240501-202123-AutoExport.xlsx
./data\HidexAMG-Track_30min-035-20240501-203711-AutoExport.xlsx
./data\HidexAMG-Track_30min-035-20240501-205301-AutoExport.xlsx
./data\HidexAMG-Track_30min-035-20240501-210850-AutoExport.xlsx
./data\HidexAMG-Track_30min-035-20240501-212438-AutoExport.xlsx
./data\HidexAMG-Track_30min-035-20240501-214026-AutoExport.xlsx
./data\HidexAMG-Track_30min-035-20240501-215615-AutoExport.xlsx
./data\HidexAMG-Track_30min-035-20240501-221204-AutoExport.xlsx
./data\HidexAMG-Track_30min-035-20240501-222752-AutoExport.xlsx
./data\HidexAMG-Track_30min-035-20240501

array(['2024-05-01T18:47:33', '2024-05-01T18:49:03',
       '2024-05-01T18:50:33', '2024-05-01T18:52:02',
       '2024-05-01T18:53:31', '2024-05-01T18:55:00',
       '2024-05-01T18:56:29', '2024-05-01T18:57:57',
       '2024-05-01T18:59:27', '2024-05-01T19:00:56',
       '2024-05-01T19:03:18', '2024-05-01T19:04:47',
       '2024-05-01T19:06:17', '2024-05-01T19:07:46',
       '2024-05-01T19:09:15', '2024-05-01T19:10:43',
       '2024-05-01T19:12:12', '2024-05-01T19:13:41',
       '2024-05-01T19:15:10', '2024-05-01T19:16:39',
       '2024-05-01T19:19:06', '2024-05-01T19:20:36',
       '2024-05-01T19:22:05', '2024-05-01T19:23:35',
       '2024-05-01T19:25:03', '2024-05-01T19:26:33',
       '2024-05-01T19:28:01', '2024-05-01T19:29:30',
       '2024-05-01T19:30:59', '2024-05-01T19:32:28',
       '2024-05-01T19:34:55', '2024-05-01T19:36:25',
       '2024-05-01T19:37:54', '2024-05-01T19:39:23',
       '2024-05-01T19:40:53', '2024-05-01T19:42:22',
       '2024-05-01T19:43:50', '2024-05-01T19:4

In [32]:
# bounds calculated in http://localhost:8888/lab/tree/efficiency/low_activity/efficiency_low_activity.ipynb
p1_start, p1_end, p2_start, p2_end =  389, 634, 1007, 1038
plt.figure()
counts = np.sum(spectra[:, p1_start:p1_end+1], axis=1) + np.sum(spectra[:, p2_start:p2_end+1], axis=1) # add up the counts in both peaks

# remove activities higher than deadtime factor 1.1
mask = np.loadtxt("deadtime_mask.txt")
counts *= mask # gives nan values where the deadtime factor too high. Calculated in efficiency_set4.ipynb

# do deadtime correction
deadtime = np.loadtxt("deadtime_factors.txt")
counts *= deadtime

print(np.nanmax(counts))

plt.title("Sanity check")
for i in range(0, 10):
    plt.plot(timestamps[i::n], counts[i::n], ".")

plt.show()

531702.7644444444


# Calculate efficiency

In [33]:
# collect masses
mass_data = np.genfromtxt("mass_comparison.csv", delimiter=",", skip_header=1, encoding="utf8")
m = mass_data[:,5]
N = np.shape(counts)[0]/n
masses = np.tile(m, int(N)) # match masses to number of runs

# collect times
time_in_seconds = timestamps.astype('int')

# calculate the concentrations in the samples in CPM/ml (assuming 1g/ml)
conc = counts/masses

c = 0.0679/19.019 # mCi/ml, original concentration of the solution
c_time = np.datetime64('2024-05-01T17:35:00') # original time of measurement for the above concentration
c_time_in_seconds = c_time.astype('datetime64[s]').astype('int')

time_in_seconds = time_in_seconds - c_time_in_seconds # start the timer at the time of normalization

def decay(A0, t, h=109.7*60):
    # input t must be in seconds
    return A0*2**(-t/h)

# this is the theoretical concentration of radiation for each measurement in mCi/ml
ref = decay(c, time_in_seconds)


# calculate the efficiency
efficiency = conc/ref # in CPM/mCi
efficiency = efficiency * 1000 / (3.7e10 * 60) *100 # transform the cpm such that we get mCi/mCi

ref_activity = ref*masses*1000 # theoretical activity in tubes in uCi

In [38]:
# load previous data
ref_activity_old = np.loadtxt("reference_activity_400-600.txt")
efficiency_old = np.loadtxt("efficiency_400-600.txt")
ref_timestamps = np.loadtxt("timestamps_400-600.txt", dtype='datetime64[s]')

# plotting
plt.figure(figsize=(8,6))
plt.title("Efficiency of Hidex AMG")
plt.ylabel("Efficiency [%]")
plt.xlabel(r"Theoretical activity [$\mu$Ci]")

#for i in range(0, 10):
#    plt.plot(ref_activity[i::n], efficiency[i::n], ".", label=str(masses[i]) + " mL")

plt.plot(ref_activity, efficiency, ".", label="Optimized energy windows")
    
#plt.xlim(0, 0.001)
plt.plot(ref_activity_old, efficiency_old, "k.", label="400-600 keV window")
#plt.xscale('log')
plt.grid()
x = ref_activity[np.isfinite(efficiency)][:500]
y = efficiency[np.isfinite(efficiency)][:500]

b, cov = np.polyfit(x, y, 1, cov = True)
print(b)
print(cov)
plt.plot(x, x*b[0] + b[1], color='k', label="Linear least-squares fit")
perr = np.sqrt(np.diag(cov))

x = np.linspace(min(x), max(x), 126)
y_plus = (b[0]+perr[0])*x + b[1]+perr[1]
y_minus = (b[0]-perr[0])*x + b[1]-perr[1]
plt.fill_between(x, y_minus, y_plus, alpha=0.2, color='k')

plt.legend()
plt.show()

[-2.19605189 38.95954834]
[[ 0.01509132 -0.0019156 ]
 [-0.0019156   0.00061728]]
